In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [2]:
# Import API key
from api_keys import g_key
if g_key == "YOUR KEY HERE!":
    import sys
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import g_key

Connect to csv file and create dataframe

In [3]:
csv_path = "Datasets/ufo_location_shape.csv"
ufo_df = pd.read_csv(csv_path)
ufo_df['Event.Date'] = pd.to_datetime(ufo_df['Event.Date'], format="%m/%d/%Y")
ufo_df.head()

,Event.Date,Shape,Location,State,Country,Source,USA,Unnamed: 7
0,2016-06-18,Boomerang/V-Shaped,South Barrington,IL,USA,NUFORC,1,NaN
1,2016-06-17,Boomerang/V-Shaped,Kuna,ID,USA,NUFORC,1,NaN
2,2016-05-30,Boomerang/V-Shaped,Lake Stevens,WA,USA,NUFORC,1,NaN
3,2016-05-27,Boomerang/V-Shaped,Gerber,CA,USA,NUFORC,1,NaN
4,2016-05-24,Boomerang/V-Shaped,Camdenton,MO,USA,NUFORC,1,NaN


Check how many rows and columns are in the dataframe

In [4]:
ufo_df.shape

(3646, 8)

Check how many unique countries are in the dataset

In [5]:
countries = ufo_df['Country'].unique()
print(countries)

['USA ' 'Canada ' 'India ' 'Norway ' 'PA ' 'Wales/UK ' 'China '
 'England/UK ' nan 'Philippines ' 'Iran ' 'Unknown ' 'Mauritius '
 'Australia ' 'Dominican Republic ' 'Gibralter ' 'New Zealand '
 'South Africa ' 'Kenya ' 'Israel ' 'Japan ' 'Mexico ' 'Netherlands '
 'Puerto Rico ' 'MO ' 'OH ' 'Lithuania ' 'Indonesia ' 'Portugal '
 'Ireland ' 'Thailand ' 'Brazil ' 'Chile ' 'CO ' 'Cambodia ' 'Pakistan '
 'The Bahamas (Bermuda triangle) ' 'Sweden ' 'NC ' 'NY ' 'OR ' 'Finland '
 'Turkey ' 'Italy ' 'Croatia ' 'KS ' 'WI ' 'NV ' 'Argentina ' 'Spain '
 'South Australia ' 'Germany ' 'Tonga ']


Filter for only USA

In [6]:
ufo_df_country = ufo_df.loc[(ufo_df['Country']=='USA ')]
ufo_df_country.head()

,Event.Date,Shape,Location,State,Country,Source,USA,Unnamed: 7
0,2016-06-18,Boomerang/V-Shaped,South Barrington,IL,USA,NUFORC,1,NaN
1,2016-06-17,Boomerang/V-Shaped,Kuna,ID,USA,NUFORC,1,NaN
2,2016-05-30,Boomerang/V-Shaped,Lake Stevens,WA,USA,NUFORC,1,NaN
3,2016-05-27,Boomerang/V-Shaped,Gerber,CA,USA,NUFORC,1,NaN
4,2016-05-24,Boomerang/V-Shaped,Camdenton,MO,USA,NUFORC,1,NaN


Verify rows were removed

In [7]:
ufo_df_country.shape

(3427, 8)

Drop unneeded columns and check amount of columns/rows

In [8]:
ufo_df_dropped = ufo_df_country.drop(['Unnamed: 7', 'USA', 'Source'], axis = 1)
ufo_df_dropped = ufo_df_dropped.rename(columns={'Event.Date': 'Date'})

In [9]:
ufo_df_dropped.shape

(3427, 5)

Drop NaN rows and check rows counts

In [10]:
ufo_df_cleaned = ufo_df_dropped.dropna()
ufo_df_cleaned.head()


,Date,Shape,Location,State,Country
0,2016-06-18,Boomerang/V-Shaped,South Barrington,IL,USA
1,2016-06-17,Boomerang/V-Shaped,Kuna,ID,USA
2,2016-05-30,Boomerang/V-Shaped,Lake Stevens,WA,USA
3,2016-05-27,Boomerang/V-Shaped,Gerber,CA,USA
4,2016-05-24,Boomerang/V-Shaped,Camdenton,MO,USA


In [11]:
ufo_df_cleaned.shape

(3414, 5)

In [12]:
ufo_df_cleaned.dtypes

Date        datetime64[ns]
Shape               object
Location            object
State               object
Country             object
dtype: object

Filter for 2016 and check row counts

In [13]:
initial_date = "2016-01-01"
end_date = "2016-12-31"
ufo_df_date = ufo_df_cleaned.loc[(ufo_df_cleaned['Date']>=initial_date) & (ufo_df_cleaned['Date']<= end_date)]
ufo_df_date_sorted = ufo_df_date.sort_values(by=['Date'])
ufo_df_date_sorted = ufo_df_date_sorted.reset_index(drop=True)
ufo_df_date_sorted.head()

,Date,Shape,Location,State,Country
0,2016-01-01,Unknown,Ontario,CA,USA
1,2016-01-01,Diamond,Longmont,CO,USA
2,2016-01-01,Lights Only,Auburn,WA,USA
3,2016-01-01,Lights Only,Smyrna,GA,USA
4,2016-01-01,Circle,Richland,WA,USA


In [14]:
ufo_df_date_sorted.shape

(2634, 5)

In [15]:
ufo_df_date_sorted['Location'] = ufo_df_date_sorted['Location'].str.replace(r"\(.*\)","")
ufo_df_date_sorted.head()

,Date,Shape,Location,State,Country
0,2016-01-01,Unknown,Ontario,CA,USA
1,2016-01-01,Diamond,Longmont,CO,USA
2,2016-01-01,Lights Only,Auburn,WA,USA
3,2016-01-01,Lights Only,Smyrna,GA,USA
4,2016-01-01,Circle,Richland,WA,USA


In [16]:
ufo_df_date_sorted.tail()

,Date,Shape,Location,State,Country
2629,2016-07-22,Lights Only,Sedona,AZ,USA
2630,2016-07-22,Oval,Billings,MT,USA
2631,2016-07-22,Circle,Bangor,ME,USA
2632,2016-07-22,Lights Only,Las Vegas,NV,USA
2633,2016-07-22,Lights Only,Renton,WA,USA


Add latitude and longitude columns

In [17]:
ufo_df_date_sorted['Lat']=""
ufo_df_date_sorted['Lng']=""
ufo_df_date_sorted.head()

,Date,Shape,Location,State,Country,Lat,Lng
0,2016-01-01,Unknown,Ontario,CA,USA,,
1,2016-01-01,Diamond,Longmont,CO,USA,,
2,2016-01-01,Lights Only,Auburn,WA,USA,,
3,2016-01-01,Lights Only,Smyrna,GA,USA,,
4,2016-01-01,Circle,Richland,WA,USA,,


Find the unique locations and try to clean for unnecessary characters

In [18]:
locations = ufo_df_date_sorted['Location'].unique()
locations

array(['Ontario ', 'Longmont ', 'Auburn ', ..., 'Spanish Fort ',
       'Sedona ', 'Renton '], dtype=object)

In [19]:
ufo_df_date_sorted['Location'] = ufo_df_date_sorted['Location'].str.replace(r"\(.*\)","")
ufo_df_date_sorted

,Date,Shape,Location,State,Country,Lat,Lng
0,2016-01-01,Unknown,Ontario,CA,USA,,
1,2016-01-01,Diamond,Longmont,CO,USA,,
2,2016-01-01,Lights Only,Auburn,WA,USA,,
3,2016-01-01,Lights Only,Smyrna,GA,USA,,
4,2016-01-01,Circle,Richland,WA,USA,,
...,...,...,...,...,...,...,...
2629,2016-07-22,Lights Only,Sedona,AZ,USA,,
2630,2016-07-22,Oval,Billings,MT,USA,,
2631,2016-07-22,Circle,Bangor,ME,USA,,
2632,2016-07-22,Lights Only,Las Vegas,NV,USA,,


In [20]:
locations = ufo_df_date_sorted['Location'].unique()
locations

array(['Ontario ', 'Longmont ', 'Auburn ', ..., 'Spanish Fort ',
       'Sedona ', 'Renton '], dtype=object)

Add the latitude and longitude for the location of the sightings from google maps api.

In [21]:
 # create a params dict that will be updated with new city each iteration
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in ufo_df_date_sorted.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        city = row['Location']
        state = row['State']

        # update address key value
        params['address'] = f"{city},{state}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)
        
        # convert to json
        cities_lat_lng = cities_lat_lng.json()

        ufo_df_date_sorted.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        ufo_df_date_sorted.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

    except:
        print(f'City not found. Skipping...')
    time.sleep(0.1)
    
print("Process finished!")
# # Visualize to confirm lat lng appear
ufo_df_date_sorted.head()

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
Process finished!


,Date,Shape,Location,State,Country,Lat,Lng
0,2016-01-01,Unknown,Ontario,CA,USA,34.0633,-117.651
1,2016-01-01,Diamond,Longmont,CO,USA,40.1672,-105.102
2,2016-01-01,Lights Only,Auburn,WA,USA,47.3073,-122.228
3,2016-01-01,Lights Only,Smyrna,GA,USA,33.884,-84.5144
4,2016-01-01,Circle,Richland,WA,USA,46.2804,-119.275


In [22]:
ufo_df_date_sorted

,Date,Shape,Location,State,Country,Lat,Lng
0,2016-01-01,Unknown,Ontario,CA,USA,34.0633,-117.651
1,2016-01-01,Diamond,Longmont,CO,USA,40.1672,-105.102
2,2016-01-01,Lights Only,Auburn,WA,USA,47.3073,-122.228
3,2016-01-01,Lights Only,Smyrna,GA,USA,33.884,-84.5144
4,2016-01-01,Circle,Richland,WA,USA,46.2804,-119.275
...,...,...,...,...,...,...,...
2629,2016-07-22,Lights Only,Sedona,AZ,USA,34.8697,-111.761
2630,2016-07-22,Oval,Billings,MT,USA,45.7833,-108.501
2631,2016-07-22,Circle,Bangor,ME,USA,44.8016,-68.7712
2632,2016-07-22,Lights Only,Las Vegas,NV,USA,36.1699,-115.14


Add columns for closest airports and do nearby search for airports within 50 miles.

In [23]:
ufo_df_date_sorted['Closest Airport'] = ""
ufo_df_date_sorted['Closest Airport Address'] = ""
ufo_df_date_sorted.head()

,Date,Shape,Location,State,Country,Lat,Lng,Closest Airport,Closest Airport Address
0,2016-01-01,Unknown,Ontario,CA,USA,34.0633,-117.651,,
1,2016-01-01,Diamond,Longmont,CO,USA,40.1672,-105.102,,
2,2016-01-01,Lights Only,Auburn,WA,USA,47.3073,-122.228,,
3,2016-01-01,Lights Only,Smyrna,GA,USA,33.884,-84.5144,,
4,2016-01-01,Circle,Richland,WA,USA,46.2804,-119.275,,


In [24]:
 # create a params dict that will be updated with new city each iteration
params = {
    "radius": 80467.2,
    "types": "airport",
    "key": g_key
}
# Loop through the cities_pd and run a lat/long search for each city
for index, row in ufo_df_date_sorted.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ufo_df_date_sorted.loc[index,'Closest Airport'] = name_address["results"][0]["name"]
        ufo_df_date_sorted.loc[index,'Closest Airport Address'] = name_address["results"][0]["vicinity"]
    #     cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print(f'{index}: Missing field/result... skipping.')
    time.sleep(0.1)
  

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [25]:
ufo_df_final = ufo_df_date_sorted.copy()

In [26]:
ufo_df_final

,Date,Shape,Location,State,Country,Lat,Lng,Closest Airport,Closest Airport Address
0,2016-01-01,Unknown,Ontario,CA,USA,34.0633,-117.651,,
1,2016-01-01,Diamond,Longmont,CO,USA,40.1672,-105.102,Boulder Municipal Airport-BDU,"3327 Airport Road, Boulder"
2,2016-01-01,Lights Only,Auburn,WA,USA,47.3073,-122.228,Seattle-Tacoma International Airport,"17801 International Boulevard, Seattle"
3,2016-01-01,Lights Only,Smyrna,GA,USA,33.884,-84.5144,Hartsfield-Jackson Atlanta International Airport,"6000 North Terminal Parkway, Atlanta"
4,2016-01-01,Circle,Richland,WA,USA,46.2804,-119.275,Tri-Cities Airport,"3601 North 20th Avenue, Pasco"
...,...,...,...,...,...,...,...,...,...
2629,2016-07-22,Lights Only,Sedona,AZ,USA,34.8697,-111.761,Flagstaff Pulliam Airport,"6200 South Pulliam Drive, Flagstaff"
2630,2016-07-22,Oval,Billings,MT,USA,45.7833,-108.501,Terminal,"1901 Terminal Circle, Billings"
2631,2016-07-22,Circle,Bangor,ME,USA,44.8016,-68.7712,Lucky Landing Marina and Seaplane Base,Glenburn
2632,2016-07-22,Lights Only,Las Vegas,NV,USA,36.1699,-115.14,Big Land Desert City,"160 Las Vegas Boulevard North, Las Vegas"


Add columns for closest airports and do nearby search for military installation within 50 miles.

In [27]:
ufo_df_final['Closest Military Base'] = ""
ufo_df_final['Closest Military Base Address'] = ""
ufo_df_final.head()

,Date,Shape,Location,State,Country,Lat,Lng,Closest Airport,Closest Airport Address,Closest Military Base,Closest Military Base Address
0,2016-01-01,Unknown,Ontario,CA,USA,34.0633,-117.651,,,,
1,2016-01-01,Diamond,Longmont,CO,USA,40.1672,-105.102,Boulder Municipal Airport-BDU,"3327 Airport Road, Boulder",,
2,2016-01-01,Lights Only,Auburn,WA,USA,47.3073,-122.228,Seattle-Tacoma International Airport,"17801 International Boulevard, Seattle",,
3,2016-01-01,Lights Only,Smyrna,GA,USA,33.884,-84.5144,Hartsfield-Jackson Atlanta International Airport,"6000 North Terminal Parkway, Atlanta",,
4,2016-01-01,Circle,Richland,WA,USA,46.2804,-119.275,Tri-Cities Airport,"3601 North 20th Avenue, Pasco",,


In [37]:
 # create a params dict that will be updated with new city each iteration
params = {
    "radius": 80467.2,
    "types": "bases",
    "keyword": "military",
    "key": g_key
}
# Loop through the cities_pd and run a lat/long search for each city
for index, row in ufo_df_final.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ufo_df_final.loc[index,'Closest Military Base']= name_address["results"][0]["name"]
        ufo_df_final.loc[index,'Closest Military Base Address'] = name_address["results"][0]["vicinity"]

    except (KeyError, IndexError):
        print(f"{index}: Missing field/result... skipping.")
    time.sleep(0.1)

Missing field/result... skipping.


In [2]:
ufo_df_final

NameError: name 'ufo_df_final' is not defined

In [1]:
ufo_df_final.to_csv("ufo_airports.csv")

NameError: name 'ufo_df_final' is not defined